In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

In [88]:

class Bilinear_Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim):
        super(Bilinear_Attention, self).__init__()
        self.W = nn.Linear(decoder_dim, encoder_dim, bias = False)
        self.fc = nn.Linear(decoder_dim + encoder_dim, decoder_dim)
        
    def forward(self, encoder_out, decoder_hidden):
        decoder_attn = self.W(decoder_hidden)
        score = torch.matmul(decoder_attn, encoder_out.transpose(1,2))
        att_weights = torch.softmax(score, dim = 1)
        att = torch.sum(torch.matmul(att_weights,encoder_out), dim = 1)
        
        new_hidden = torch.cat((decoder_hidden, att.unsqueeze(0).repeat(decoder_hidden.shape[0],1,1)) , dim = 2)
        new_hidden = self.fc(new_hidden)
        new_hidden = torch.tanh(new_hidden)
        return new_hidden


In [91]:
encoder_out = torch.ones(2,3,5)
dec_hid = torch.ones(2,2,5)

att = Bilinear_Attention(5,5)

print(att.forward(encoder_out, dec_hid))

torch.Size([2, 2, 5])
torch.Size([2, 5, 3])
tensor([[[-0.1130, -0.5968, -0.3299,  0.9539, -0.4031],
         [-0.1130, -0.5968, -0.3299,  0.9539, -0.4031]],

        [[-0.1130, -0.5968, -0.3299,  0.9539, -0.4031],
         [-0.1130, -0.5968, -0.3299,  0.9539, -0.4031]]],
       grad_fn=<TanhBackward0>)
